# Reserva de Hotel com Middleware de Membro Prioritário

Este notebook demonstra **middleware baseado em função** usando o Microsoft Agent Framework. Construímos sobre o exemplo de fluxo de trabalho condicional adicionando uma camada de middleware que confere privilégios especiais aos membros prioritários.

## O Que Vai Aprender:
1. **Middleware Baseado em Função**: Interceptar e modificar resultados de funções
2. **Acesso ao Contexto**: Ler e modificar `context.result` após a execução
3. **Implementação da Lógica de Negócio**: Benefícios para membros prioritários
4. **Sobrescrita de Resultado**: Alterar os resultados da função com base no estado do utilizador
5. **Mesmo Fluxo, Resultados Diferentes**: Mudanças de comportamento impulsionadas pelo middleware

## Arquitetura do Fluxo de Trabalho com Middleware:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Diferença Principal em Relação ao Fluxo de Trabalho Condicional:

**Sem Middleware** (14-conditional-workflow.ipynb):
- Paris não tem quartos → Redirecionar para alternative_agent

**Com Middleware** (este notebook):
- Utilizador regular + Paris → Sem quartos → Redirecionar para alternative_agent
- Utilizador prioritário + Paris → 🌟 Middleware sobrepõe! → Disponível → Redirecionar para booking_agent

## Pré-Requisitos:
- Microsoft Agent Framework instalado
- Compreensão de fluxos de trabalho condicionais (ver 14-conditional-workflow.ipynb)
- Token GitHub ou chave API OpenAI
- Conhecimentos básicos de padrões de middleware


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: Definir Modelos Pydantic para Saídas Estruturadas

Estes modelos definem o **esquema** que os agentes irão devolver. Adicionámos um campo `priority_override` para rastrear quando o middleware modifica o resultado de disponibilidade.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: Definir Base de Dados de Membros Prioritários

Para esta demonstração, vamos simular uma base de dados de membros prioritários. Em produção, isto consultaria uma base de dados ou API real.

**Membros Prioritários:**
- `alice@example.com` - membro VIP
- `bob@example.com` - membro Premium  
- `priority_user` - conta de teste


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Passo 3: Criar a Ferramenta de Reserva de Hotel

Igual ao fluxo de trabalho condicional, mas agora será interceptado por middleware!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 4: 🌟 Criar Middleware de Verificação de Prioridade (O RECURSO PRINCIPAL!)

Esta é a **funcionalidade principal** deste notebook. O middleware:

1. **Intercepte** a chamada da função hotel_booking
2. **Execute** a função normalmente chamando `next(context)`
3. **Inspecione** o resultado em `context.result`
4. **Substitua** o resultado se o utilizador for prioritário e não houver quartos disponíveis
5. **Retorne** o resultado modificado de volta ao agente

**Padrão Chave:**
```python
async def my_middleware(context, next):
    await next(context)  # Executar função
    # Agora context.result contém a saída da função
    if some_condition:
        context.result = new_value  # Sobrescrever!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Passo 5: Definir Funções de Condição para o Roteamento

Mesmas funções de condição do fluxo de trabalho condicional - elas inspecionam a saída estruturada para determinar o roteamento.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Passo 6: Criar Executor de Exibição Personalizado

Mesmo executor de antes - exibe o resultado final do fluxo de trabalho.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Passo 7: Carregar Variáveis de Ambiente

Configure o cliente LLM (Modelos do GitHub ou OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Passo 8: Criar Agentes de IA com Middleware

**DIFERENÇA CHAVE:** Ao criar o availability_agent, passamos o parâmetro `middleware`!

É assim que injetamos o priority_check_middleware na pipeline de invocação da função do agente.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Passo 9: Construir o Fluxo de Trabalho

Mesma estrutura de fluxo de trabalho de antes - encaminhamento condicional baseado na disponibilidade.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 10: Caso de Teste 1 - Utilizador Regular em Paris (Sem Substituição)

Um utilizador regular tenta reservar Paris → Sem quartos → Redireciona para alternative_agent


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Passo 11: Caso de Teste 2 - 🌟 Utilizador Prioritário em Paris (COM Override!)

Um membro prioritário tenta reservar Paris → Sem quartos inicialmente → 🌟 Middleware faz override! → Encaminha para booking_agent

**Esta é a demonstração chave do poder do middleware!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Passo 12: Caso de Teste 3 - Utilizador Prioritário em Estocolmo (Já Disponível)

O utilizador prioritário tenta Estocolmo → Quartos disponíveis → Não é necessário sobrepor → Encaminha para booking_agent

Isto mostra que o middleware só atua quando é necessário!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Principais Conclusões e Conceitos de Middleware

### ✅ O que Aprendeste:

#### **1. Padrão Middleware Baseado em Funções**

O middleware intercepta chamadas de funções utilizando uma função async simples:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Antes da execução da função
    print("Intercepting...")
    
    # Executar a função
    await next(context)
    
    # Após a execução da função - inspecionar resultado
    if context.result:
        # Modificar o resultado se necessário
        context.result = modified_value
```

#### **2. Acesso ao Contexto e Sobrescrita de Resultado**

- `context.function` - Aceder à função que está a ser chamada
- `context.arguments` - Ler os argumentos da função
- `context.kwargs` - Aceder a parâmetros adicionais
- `await next(context)` - Executar a função
- `context.result` - Ler/modificar a saída da função

#### **3. Implementação da Lógica de Negócio**

O nosso middleware implementa benefícios para membros prioritários:
- **Utilizadores normais**: Sem alterações, fluxo padrão
- **Utilizadores prioritários**: Sobrescreve "sem disponibilidade" → "disponível"
- **Lógica condicional**: Só sobrescreve quando necessário

#### **4. Mesmo Fluxo de Trabalho, Resultados Diferentes**

O poder do middleware:
- ✅ Sem alterações na estrutura do fluxo de trabalho
- ✅ Sem alterações na função da ferramenta
- ✅ Sem alterações na lógica de roteamento condicional
- ✅ Apenas middleware → Comportamento diferente!

### 🚀 Aplicações no Mundo Real:

1. **Funcionalidades VIP/Premium**
   - Sobrescrever limites de taxa para utilizadores premium
   - Fornecer acesso prioritário a recursos
   - Desbloquear funcionalidades premium dinamicamente

2. **Testes A/B**
   - Direcionar utilizadores para implementações diferentes
   - Testar novas funcionalidades com utilizadores específicos
   - Lançamentos graduais de funcionalidades

3. **Segurança e Conformidade**
   - Auditar chamadas de função
   - Bloquear operações sensíveis
   - Aplicar regras de negócio

4. **Otimização de Desempenho**
   - Guardar em cache resultados para utilizadores específicos
   - Evitar operações dispendiosas quando possível
   - Alocação dinâmica de recursos

5. **Manipulação de Erros e Tentativas**
   - Capturar e tratar erros de forma elegante
   - Implementar lógica de tentativas
   - Recorrer a implementações alternativas

6. **Registo e Monitorização**
   - Rastrear tempos de execução de funções
   - Registar parâmetros e resultados
   - Monitorizar padrões de uso

### 🔑 Diferenças-Chave em Relação aos Decoradores:

| Funcionalidade | Decorador | Middleware |
|----------------|-----------|------------|
| **Âmbito** | Função única | Todas as funções do agente |
| **Flexibilidade** | Fixo na definição | Dinâmico em tempo de execução |
| **Contexto** | Limitado | Contexto completo do agente |
| **Composição** | Múltiplos decoradores | Pipeline de middleware |
| **Consciente do Agente** | Não | Sim (acesso ao estado do agente) |

### 📚 Quando Usar Middleware:

✅ **Usa middleware quando:**
- Precisas modificar comportamento com base no estado do utilizador/sessão
- Queres aplicar lógica a múltiplas funções
- Necessitas acesso ao contexto a nível de agente
- Estás a implementar preocupações transversais (registo, autenticação, etc.)

❌ **Não uses middleware quando:**
- Validação simples de entrada (usa Pydantic)
- Lógica específica de função (mantém na função)
- Modificações pontuais (altera só a função)

### 🎓 Padrões Avançados:

```python
# Múltiplos middleware (a ordem de execução importa!)
middleware=[
    logging_middleware,      # Regista primeiro
    auth_middleware,         # Depois verifica a autenticação
    cache_middleware,        # Depois verifica a cache
    rate_limit_middleware,   # Depois limita a taxa
    priority_check_middleware  # Finalmente verifica a prioridade
]

# Execução condicional do middleware
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modificar o resultado
    else:
        # Saltar a execução completamente
        context.result = cached_value
```

### 🔗 Conceitos Relacionados:

- **Middleware do Agente**: Intercepta chamadas agent.run()
- **Middleware de Função**: Intercepta chamadas de funções de ferramentas (o que usamos!)
- **Pipeline de Middleware**: Cadeia de middleware que executa por ordem
- **Propagação de Contexto**: Passar estado pela cadeia de middleware


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:
Este documento foi traduzido utilizando o serviço de tradução automática [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos por garantir a precisão, por favor tenha em atenção que traduções automáticas podem conter erros ou imprecisões. O documento original no seu idioma nativo deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se tradução profissional humana. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
